<h1> A1 Regression Analysis - Birthweight </h1>

In our first assignment in Machine Learning, we try to predict the birthweight of newborn babies and give advice to the mothers what to do in pregnancy. There are several factors playing a role for the weight, for example the amount of cigarettes smoked by the mother. The factors and impacts are explained later on.
Our model cannot predict the birthweight perfectly, as other factors play an important role too. The authors in <a href="https://www.marsden-weighing.co.uk/blog/factors-affect-baby-birth-weight">this website</a> state that for example heights of mother and father, the number of babies and length of pregnancy are important factors for the birthweight. These information are not delivered in the data.

<h2> Code for regression analysis </h2>

Run this code for the regression analysis on the baby weights.

In [ ]:
# import used packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# import packages to do different types of regression
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ARDRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split


# list for the column names
cols = ['mother_age', 'mother_education', 'begin_prenatal_care', 'number_prenatal_visits',
       'father_age', 'father_education', 'one_minute_apgar', 'five_minute_apgar',
       'cigarettes', 'drinks', 'baby_male', 'mother_white', 'mother_black', 'mother_other',
       'father_white', 'father_black', 'father_other', 'bodyweight']

# read in data
file = 'birthweight_low.xlsx'
df = pd.read_excel(io         = file,
                  header      = 0,
                  sheet_name  = 0,
                  names       = cols)

# imputing missing values
df['mother_education'] = df['mother_education'].fillna(int(df['mother_education'].mean(axis=0)))
df['father_education'] = df['father_education'].fillna(int(df['father_education'].mean(axis=0)))
df['number_prenatal_visits'] = df['number_prenatal_visits'].fillna(int(df['number_prenatal_visits'].mode()[0]))

# creating log column for bodyweight
df['log_bodyweight'] = np.log(df['bodyweight'])

# calculations on mother's age --> create log column and column if mother is over 40
df['log_mother_age'] = np.log(df['mother_age'])
df['mother_over_40'] = pd.cut(df['mother_age'], [-1, 40, 100], include_lowest=True, labels=[0,1]).astype(int)

# calculations on father's age --> create log column and column if mother is over 40
df['log_father_age'] = np.log(df['father_age'])
df['father_over_40'] = pd.cut(df['father_age'], [-1, 40, 100], include_lowest=True, labels=[0,1]).astype(int)

# calculate additional columns for the age --> sum of the ages and if both are over 40
df['age_total'] = df['mother_age'] + df['father_age']
df['age_mult'] = df['mother_age'] * df['father_age']
df['both_over_40'] = df['mother_over_40'] * df['father_over_40']

# creating log column for mother's education
df['log_mother_education'] = np.log(df['mother_education'])

# creating column for checking if mother has been to college or above
df['mother_college'] = pd.cut(df['mother_education'], [-1, 12, 100], include_lowest=True, labels=[0,1]).astype(int)

# creating log column for father's education
df['log_father_education'] = np.log(df['father_education'])

# creating column for checking if father has been to college or above
df['father_college'] = pd.cut(df['father_education'], [-1, 12, 100], include_lowest=True, labels=[0,1]).astype(int)

# creating column for checking if prenatal care began late
df['begin_prenatal_care_late'] = pd.cut(df['begin_prenatal_care'], [-1, 2, 100], include_lowest=True, labels=[0,1]).astype(int)

# creating column for checking if prenatal care visits were too low
df['number_prenatal_visits_low'] = pd.cut(df['number_prenatal_visits'], [-1, 11, 100], include_lowest=True, labels=[1,0]).astype(int)

# smoking and drinking together 
df['sins_total'] = df['cigarettes'] + df['drinks']
df['sins_mult'] = df['cigarettes'] * df['drinks']


# create a column for mother's race
df['mother_race'] = 0
for index, value in df.iterrows():
    # mother white
    if df.loc[index, 'mother_white'] == 1:
        df.loc[index, 'mother_race'] = 'White'
    # mother black
    elif df.loc[index, 'mother_black'] == 1:
        df.loc[index, 'mother_race'] = 'Black'
    # mother white
    elif df.loc[index, 'mother_other'] == 1:
        df.loc[index, 'mother_race'] = 'Other'
    else:
        print('No race found')
        
# create a column for father's race
df['father_race'] = 0
for index, value in df.iterrows():
    # father white
    if df.loc[index, 'father_white'] == 1:
        df.loc[index, 'father_race'] = 'White'
    # father black
    elif df.loc[index, 'father_black'] == 1:
        df.loc[index, 'father_race'] = 'Black'
    # mother white
    elif df.loc[index, 'father_other'] == 1:
        df.loc[index, 'father_race'] = 'Other'
    else:
        print('No race found')
        
df['both_white'] = df['mother_white']*df['father_white']
df['both_black'] = df['mother_black']*df['father_black']
df['both_other'] = df['mother_other']*df['father_other']
df['is_same_race'] = df['both_white']+df['both_black']+df['both_other']


# making a copy of the df
df_explanatory = df.copy()


# dropping bodyweight and log_bodyweight from the explanatory variable set
df_explanatory = df_explanatory.drop(['bodyweight',
                                    'log_bodyweight'], axis = 1)


# chosing used columns
X = df_explanatory[['cigarettes', 'drinks', 'mother_over_40']]
y_normal = df[['bodyweight']]
y_log = df[['log_bodyweight']]

# split into train and test sets for bodyweight and log_bodyweight
X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(X,y_normal, test_size=0.25, random_state=219)
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X,y_log, test_size=0.25, random_state=219)


# create concated df to test for p values
df_train_normal = pd.concat([X_train_normal, y_train_normal], axis = 1)

lm_best = smf.ols(formula =  """bodyweight ~ cigarettes + 
                                            drinks + 
                                            mother_over_40
                """,
                 data = df_train_normal)

#results = lm_best.fit()
#print(results.summary())


# run Linear Regression on bodyweigt and save test and train score
lr_normal = LinearRegression().fit(X_train_normal, y_train_normal)
lr_pred = lr_normal.predict(X_test_normal)
lr_normal_test = lr_normal.score(X_test_normal, y_test_normal)
lr_normal_train = lr_normal.score(X_train_normal, y_train_normal)

# run Linear Regression on log_bodyweigt and save test and train score
lr_log = LinearRegression().fit(X_train_log, y_train_log)
lr_log_test = lr_log.score(X_test_log, y_test_log)
lr_log_train = lr_log.score(X_train_log, y_train_log)

# run Lasso Regression with chosen alpha on bodyweigt and save test and train score
lasso_normal = Lasso().fit(X_train_normal, y_train_normal.values.ravel())
lasso_pred = lasso_normal.predict(X_test_normal)
lasso_normal_test = lasso_normal.score(X_test_normal, y_test_normal)
lasso_normal_train =  lasso_normal.score(X_train_normal, y_train_normal)

# run Lasso Regression with chosen alpha on log_bodyweigt and save test and train score
lasso_log = Lasso().fit(X_train_log, y_train_log.values.ravel())
lasso_log_test = lasso_log.score(X_test_log, y_test_log)
lasso_log_train = lasso_log.score(X_train_log, y_train_log)

# run ARD Regression on bodyweigt and save test and train score
ard_normal = ARDRegression().fit(X_train_normal, y_train_normal.values.ravel())
ard_pred = ard_normal.predict(X_test_normal)
ard_normal_test = ard_normal.score(X_test_normal, y_test_normal)
ard_normal_train =  ard_normal.score(X_train_normal, y_train_normal)

# run ARD Regression on log_bodyweigt and save test and train score
ard_log = ARDRegression().fit(X_train_log, y_train_log.values.ravel())
ard_log_test =  ard_log.score(X_test_log, y_test_log)
ard_log_train =  ard_log.score(X_train_log, y_train_log)

# run KNN Regression with chosen neighbors on bodyweigt and save test and train score
knn_normal = KNeighborsRegressor().fit(X_train_normal, y_train_normal.values.ravel())
knn_pred = knn_normal.predict(X_test_normal)
knn_normal_test = knn_normal.score(X_test_normal, y_test_normal)
knn_normal_train =  knn_normal.score(X_train_normal, y_train_normal)

# run KNN Regression with chosen neighbors on log_bodyweigt and save test and train score
knn_log = KNeighborsRegressor().fit(X_train_log, y_train_log.values.ravel())
knn_log_test = knn_log.score(X_test_log, y_test_log)
knn_log_train = knn_log.score(X_train_log, y_train_log)

# collect all test scores
test_scores = [lr_normal_test, lr_log_test, lasso_normal_test, lasso_log_test, ard_normal_test, ard_log_test, knn_normal_test, knn_log_test]

# collect names 
names = ['OLS normal', 'OLS log', 'Lasso normal', 'Lasso log', 'ARD normal', 'ARD log', 'KNN normal', 'KNN log']

# collect all gaps 
gaps = [abs(lr_normal_train-lr_normal_test), abs(lr_log_train-lr_log_test), abs(lasso_normal_train-lasso_normal_test),
       abs(lasso_log_train-lasso_log_test),abs(ard_normal_train-ard_normal_test), abs(ard_log_train-ard_log_test),
       abs(knn_normal_train-knn_normal_test), abs(knn_log_train-knn_log_test)]

# get the best model
best = ''
best_score = 0
# iterate through every model and substract score from gap
for i, test_score in enumerate(test_scores):
    score = test_score-gaps[i]+min(0.05, gaps[i])
    if score >best_score:
        best_score = score
        best = names[i]

print(f"""
Model          Train Score      Test Score      Train-Test-Gap
-----          -----------      ----------      --------------
OLS normal     {lr_normal_train: .4f}          {lr_normal_test: .4f}          {gaps[0].round(decimals=2)}
OLS log        {lr_log_train: .4f}          {lr_log_test: .4f}          {gaps[1].round(decimals=2)}    
Lasso normal   {lasso_normal_train: .4f}          {lasso_normal_test: .4f}          {gaps[2].round(decimals=2)}
Lasso log      {lasso_log_train: .4f}          {lasso_log_test: .4f}          {gaps[3].round(decimals=2)} 
ARD normal     {ard_normal_train: .4f}          {ard_normal_test: .4f}          {gaps[4].round(decimals=2)}
ARD log        {lasso_log_train: .4f}          {ard_log_test: .4f}          {gaps[5].round(decimals=2)} 
KNN normal     {knn_normal_train: .4f}          {knn_normal_test: .4f}          {gaps[6].round(decimals=2)}
KNN log        {knn_log_train: .4f}          {knn_log_test: .4f}          {gaps[7].round(decimals=2)}


On this analysis above, I chose the {best}-model with a final score of {(best_score*100).round(decimals=2)}%.
""")

<h2> Explanation of the factors </h2>

In this part, the columns are analyzed and simplified. To view the charts, please run the regression analysis above first, so all columns are created.

<h3> Imputing missing values </h3>

<h4> Years of education </h4>

After looking into the data, I realized there are three missing values for the mother's education and number of prenatal visits and seven missing values for the father's education. My first idea was imputing the years of education with the partners years of education if possible, as I thought partners tend to have the same years of education. But looking at the scatterplot shown below, I realized that is not true. I then decided just to go with the mean for both of the columns.



In [ ]:
# create scatterplot to compare mom's and dad's education
sns.scatterplot(x = df['mother_education'],
                    y = df['father_education'])
plt.show()


<h4> Number of prenatal visits </h4>

Looking at the histogrm below, people tend to have 12 prenatal visits. Therefor I decided to impute the missing values with 12 (the mode).

In [ ]:
# create histplot for number of prenatal visits
sns.histplot(x = df['number_prenatal_visits'])
plt.show()

<h3> y-variable: Bodyweight </h3>

The next step is looking into the distribution of the y-variable, in this case the bodyweight of the newborn babies. Analyzing the histogram of bodyweight, the distribution seems relatively normal, even though we have skewness to the left. That is why I decided to create a column for the logarithmic bodyweight too.

In [ ]:
# histplot for bodyweight
sns.histplot(data   = df,
            x      = 'bodyweight',
            kde    = True)
plt.show()

# histplot for logarithmic bodyweight
sns.histplot(data   = df,
            x      = 'log_bodyweight',
            kde    = True)

plt.show()


<h3> Explanatory variables </h3>

In this section, I am analyzing the explanatory columns. I am looking for skewness and make sense of the variables.

<h4> Mother's age </h4>
The first column I am looking into is the mother's age. You can see in the first chart that this column is skewed to the right, so I created a new column with the logarithm of the age shown in the second chart. The third chart shows the mother's age compared to the bodyweight. You can see that the bodyweight drops when the mother is 40 or older. As websites like <a href="https://www.goodto.com/family/geriatric-mother-462676">this</a> suggest, mother's are considered old over the age of 35 resulting in higher risks of pregnancy. In the third chart mother of the age 35 to 40 seem to have no problems, that is why I created a column which detects if the mother is younger or older than 40.
Additionally, we do not have any young mother under the age of 20 in our datasets. That makes it harder to draw meaningful conclusions in the end, as our data is biased.
The correlation matrix shown at the bottom suggest, that the mother's age has a negative impact on the babies weight. Also it seems that the logarithm of the bodyweight is a better choice.

In [ ]:
# create histplot for mother's age
sns.histplot(data   = df,
            x      = 'mother_age',
            kde    = True)
plt.show()

# create histplot for log of mother's age
sns.histplot(data   = df,
            x      = 'log_mother_age',
            kde    = True)
plt.show()

# instantiating an lmplot for mother's age and bodyweight
sns.lmplot(x          = 'mother_age',  
           y          = 'bodyweight',  
           hue        = 'mother_over_40', # categorical data for subsets
           legend     = False,        # supressing the legend  
           scatter    = True,     
           fit_reg    = True,     
           aspect     = 2,        
           data       = df)

# developing vertical axis line at age 40 and show plot
plt.axvline(x = 40, color = "purple", linestyle = '--')
plt.show()

# create correlation matrix for all mother's age columns with the bodyweight columns
df.corr().loc[['mother_age', 'log_mother_age', 'mother_over_40'], ['bodyweight', 'log_bodyweight']]

<h4> Father's age </h4>
After the mother's age, I am now analyzing the father's age in the same manner. Again we see a skewed histogram, this time to the right. That is why I again created a logarithmic column of the father's age. After doing so, the second graph looks relatively normal.
Again, the father's age has an impact on the health of the baby. <a href="https://med.stanford.edu/news/all-news/2018/10/older-fathers-associated-with-increased-birth-risks.html">Hanae Armitage</a> shows, that fathers who are 45 or older were 14 percent more likely to have a child born prematurely, and men 50 or older were 28 percent more likely to have a child that required admission to the neonatal intensive care unit. The author shows that 35 seems to be the critical age, but again I set the border to 40 and older. The correlation matrix does not show such a big impact compared to the mother's age, however it seems to be important if the dad is over 40.
As the age seems to play an important role, I also created columns of the total age of both parents (mother's age + father's age) and if both parents are over 40. The correlation matrix suggests, that the total age of both parents is important for the babies weight.

In [ ]:
# create histplot for fathers's age
sns.histplot(data   = df,
            x      = 'father_age',
            kde    = True)
plt.show()

# create histplot for log of father's age
sns.histplot(data   = df,
            x      = 'log_father_age',
            kde    = True)
plt.show()

# instantiating an lmplot for father's age and bodyweight
sns.lmplot(x          = 'father_age',  
           y          = 'bodyweight',  
           hue        = 'father_over_40', # categorical data for subsets
           legend     = False,        # supressing the legend  
           scatter    = True,     
           fit_reg    = True,     
           aspect     = 2,        
           data       = df)

# developing vertical axis line at age 40 and show plot
plt.axvline(x = 40, color = "purple", linestyle = '--')
plt.show()

# create correlation matrix for all fathers's age columns with the bodyweight columns
df.corr().loc[['father_age', 'log_father_age', 'father_over_40', 'age_total', 'both_over_40', 'age_mult'], 
              ['bodyweight', 'log_bodyweight']]

<h4> Mother's and Father's education </h4>

Again, the first step here is inspecting the distribution in a histogram, which resulted in creating a logarithmic column for both parents. As the boxplot shows, the mother's and father's education do not seem to have such a big impact. That is supported by the authors of <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6519047/">this paper</a>, which claim that there is a casual link between education and health, but not as significant as expected. This effect is also shown in the correlation matrix, as the values are low compared to the age. Last, I created a column for mother's and father's who have been to the college, but that does not seem to have an impact neither.

In [ ]:
# create two plots next to each other for education, log education and boxplots
fig, ax = plt.subplots(figsize = (10, 8))

# create histplot for mother's education
plt.subplot(2, 2, 1)
sns.histplot(data   = df,
            x      = 'mother_education',
            kde    = True)

# create histplot for father's education
plt.subplot(2, 2, 2)
sns.histplot(data   = df,
            x      = 'father_education',
            kde    = True)

# create histplot for log of mother's age
plt.subplot(2, 2, 3)
sns.histplot(data   = df,
            x      = 'log_father_age',
            kde    = True)

# create histplot for log of father's age
plt.subplot(2, 2, 4)
sns.histplot(data   = df,
            x      = 'log_father_age',
            kde    = True)
plt.show()

fig, ax = plt.subplots(figsize = (10, 8))
# developing a boxplot to compare mother's education and bodyweight
plt.subplot(2, 2, 1)
sns.boxplot(x    = 'mother_education',
            y    = 'bodyweight',
            data = df)

# developing a boxplot to compare mother's education and bodyweight
plt.subplot(2, 2, 2)
sns.boxplot(x    = 'father_education',
            y    = 'bodyweight',
            data = df)

# displaying the plot
plt.show()

# create correlation matrix for all mother's education columns with the bodyweight columns
df.corr().loc[['mother_education', 'mother_college','father_education', 'father_college'], 
              ['bodyweight', 'log_bodyweight']]

<h4> Begin prenatal care and number of prenatal visits </h4>

Next, I am inspecting the prenatal care. The authors on <a href="https://www.americanprogress.org/article/ensuring-healthy-births-prenatal-support/">this website</a> recommend 13 to 14 prenatal visits with a starting date not later than 10 weeks after pregnancy. In comparison to the authors claim, the number of visits and begin of prenatal care do not seem to have such a big impact on the babies weight. As proposed on the website, I checked if the prenatal care began late (after month 2), which impacts the bodyweight. As mentioned before, our women tend to go to the prenatal care 12 times, that is why I set the border of low to 11. Again, the number of visits does not seem to have an influence, only if the woman went to less times.

In [ ]:
# create two plots next to each other for begin and number of prenatal core and their boxplots
fig, ax = plt.subplots(figsize = (10, 8))


# create histplot for begin prenatal care
plt.subplot(2, 2, 1)
sns.histplot(data   = df,
            x      = 'begin_prenatal_care',
            kde    = True)

# create histplot for number prenatal care
plt.subplot(2, 2, 2)
sns.histplot(data   = df,
            x      = 'number_prenatal_visits',
            kde    = True)

# create boxplot to compare begin of prenatal care and bodyweight
plt.subplot(2, 2, 3)
sns.boxplot(x    = 'begin_prenatal_care',
            y    = 'bodyweight',
            data = df)

# create boxplot to compare number of prenatal visits and bodyweight
plt.subplot(2, 2, 4)
sns.scatterplot(x    = 'number_prenatal_visits',
            y    = 'bodyweight',
            hue = 'number_prenatal_visits_low',
            data = df)
plt.show()

# create correlation matrix for prenatal visits and begin columns with the bodyweight columns
df.corr().loc[['begin_prenatal_care', 'begin_prenatal_care_late', 'number_prenatal_visits', 'number_prenatal_visits_low'],
              ['bodyweight', 'log_bodyweight']]


<h4> Average number of cigarettes and drinks per day </h4>

Again, our data seems to be very biased, as we only have 11 non-drinkers and 9 non-smokers. Therefor it does not make sense to split the data into smokers and non-smokers and drinkers and non-drinkers. However as expected, smoking and drinking have a big impact on the babies bodyweight. The boxplots show a negative trend which is also confirmed in the correlation matrix. It also seems that drinking has a worse impact than smoking. I combined both sins in one column and added the drinks and cigarettes together, which does not seem to have an additional impact compared to the drinks alone. However, if we multiply both of the columns, the impact seems to be even bigger.


In [ ]:
# create two plots next to each other for cigarettes and drinks and their boxplots
fig, ax = plt.subplots(figsize = (10, 8))


# create histplot for cigarettes
plt.subplot(2, 2, 1)
sns.histplot(data   = df,
            x      = 'cigarettes',
            kde    = True)

# create histplot for drinks
plt.subplot(2, 2, 2)
sns.histplot(data   = df,
            x      = 'drinks',
            kde    = True)

# create boxplot to compare begin of prenatal care and bodyweight
plt.subplot(2, 2, 3)
sns.boxplot(x    = 'cigarettes',
            y    = 'bodyweight',
            data = df)

# create boxplot to compare number of prenatal visits and bodyweight
plt.subplot(2, 2, 4)
sns.boxplot(x    = 'drinks',
            y    = 'bodyweight',
            data = df)
plt.show()

# create correlation matrix for prenatal visits and begin columns with the bodyweight columns
df.corr().loc[['cigarettes', 'drinks', 'sins_total', 'sins_mult'],
              ['bodyweight', 'log_bodyweight']]

<h4> Gender </h4>

In our dataset we have 108 male and 88 female babies. The boxplots suggest that male babies tend to be a little heavier than female babies. However, the correlation matrix suggests that the gender does not play a big role on the weight.

In [ ]:
# create boxplot for gender and bodyweight
sns.boxplot(y    = 'bodyweight',
            x   = 'baby_male',
            data = df)
plt.show()

# create correlation matrix for gender with the bodyweight columns
df.corr().loc[['baby_male'],
              ['bodyweight', 'log_bodyweight']]

<h4> Race </h4>

To distinguish better between the races, I first created a two columns collecting the race (<em> mother_race</em> and <em> father_race</em>). It seems, that babies with black parents in general are heavier. Next, I inspected if race has an impact on the other factors such as age and gender. It seems that white girls tend to be smaller compared to their male brothers. But as of the few datapoints, no conclusion can be drawn in both of the correlations.

In [ ]:
# create two plots next to each other for mother and father
fig, ax = plt.subplots(figsize = (10, 8))

# create boxplot comparing mother's race and bodyweight
plt.subplot(2, 2, 1)
sns.boxplot(
            y    = 'bodyweight',
            x = 'mother_race',
            data = df)

# create boxplot comparing father's race and bodyweight
plt.subplot(2, 2, 2)
sns.boxplot(
            y    = 'bodyweight',
            x = 'father_race',
            data = df)

# create boxplot comparing male and female babies and bodyweight split by mother's race
plt.subplot(2, 2, 3)
sns.boxplot(y    = 'bodyweight',
            x   = 'baby_male',
            hue = 'mother_race',
            data = df)

# create boxplot comparing male and female babies and bodyweight split by  fathers race
plt.subplot(2, 2, 4)
sns.boxplot(y    = 'bodyweight',
            x   = 'baby_male',
            hue = 'father_race',
            data = df)
plt.show()


# create boxplot comparing mother's race, age and bodyweight
sns.lmplot(
            y    = 'bodyweight',
            x = 'mother_age',
            hue = 'mother_race',
           legend     = True,        # supressing the legend  
           scatter    = True,     
           fit_reg    = True,     
           aspect     = 2,        
           data       = df)

plt.show()
# create boxplot comparing father's race, age and bodyweight
sns.lmplot(
            y    = 'bodyweight',
            x = 'father_age',
            hue = 'father_race',
           legend     = True,        # supressing the legend  
           scatter    = True,     
           fit_reg    = True,     
           aspect     = 2,        
           data       = df)

plt.show()

# create correlation matrix for race with the bodyweight columns
df.corr().loc[['mother_white', 'mother_black', 'mother_other', 'father_white', 'father_black', 'father_other'],
              ['bodyweight', 'log_bodyweight']]

<h4> One minute and five minute APGAR </h4>

These two factors measure the health of the baby and therefor have an impact on the bodyweight. However, the tests are done after the birth and can therefor not be considered in our regression.

In [ ]:
# create correlation matrix for one minute and five minute APGAR with the bodyweight columns
df.corr().loc[['one_minute_apgar', 'five_minute_apgar'],
              ['bodyweight', 'log_bodyweight']]